# 🎌 MyAnimeList Orderer

> **📚✨ Discover What to Watch or Read Next — Effortlessly! 🎥🌟**

Are you overwhelmed by your ever-growing anime and manga backlog on **MyAnimeList**?
Let this tool do the thinking for you! 🧠💡

With just a few clicks, you can:
- 🎯 **Sort** your list using smart filters and custom criteria
- 📊 **Enrich** entries with ratings, rankings, and descriptions
- 🌐 **Find** where to stream titles in your country via **JustWatch**
- 💾 **Export** a clean, organized table to use or share

No more endless scrolling — just clear, curated recommendations tailored to your preferences.
Let’s make your next pick the right one! ✅



## 📋 What Does This Notebook Do?

This Jupyter notebook is designed to streamline your anime or manga decision-making process using data from **MyAnimeList** and **JustWatch**.

---

### 🔍 How It Works

1. **Select a List Section**
   You choose which part of your MyAnimeList to work with (e.g., "Plan to Watch", "Watching", "Completed").

2. **Sort by Your Preferred Criteria**
   The notebook organizes your anime or manga based on the sorting criteria you specify (e.g., score, popularity, number of episodes, etc.).

3. **Data Enrichment via MyAnimeList API**
   After sorting, the notebook queries the **MyAnimeList API v2** to enrich each entry with additional details such as:
   - ✅ Title and description
   - 🎬 Number of episodes or chapters
   - 🔁 Whether it has sequels/prequels
   - ⭐ User score and global ranking

   📚 **API Reference**: [MyAnimeList API v2 Documentation](https://myanimelist.net/apiconfig/references/api/v2#section/)

4. **Streaming Availability via JustWatch**
   The notebook uses the **Simple JustWatch Python API** to fetch streaming information, showing where each title is available **in your country**.

   📚 **Library Reference**: [simple-justwatch-python-api on PyPI](https://pypi.org/project/simple-justwatch-python-api/)

---

### 📊 Output

You’ll get a clean, well-organized table with all this information, making it easier to choose what to watch or read next.


## 📦 Installing and Importing Dependencies

By running the following cell, **all required libraries will be installed and imported automatically**.
This ensures the notebook is ready to use without any manual setup.

If you encounter any errors during the process, please install the dependencies manually using `pip`.

### 📋 Required Dependencies

- `os` *(standard library, no installation needed)*
- `pandas`
- `requests`
- `simple-justwatch-python-api`

In [ ]:
#First, we need to import/install the required libraries
import os
!pip install pandas
import pandas as pd
!pip install requests
import requests
!pip install simple-justwatch-python-api
from simplejustwatchapi.justwatch import search

## 🔑 Setting Up Your MyAnimeList API Key

To access the MyAnimeList API, you need a **Client ID** (API key).
In this notebook, we load it from your system's **environment variables** to keep your credentials secure.

### 📥 How to Get Your API Key

1. Go to the official MyAnimeList developer portal.
2. Create an application.
3. Copy your **Client ID**.

📚 **Documentation**: [MyAnimeList API v2](https://myanimelist.net/apiconfig/references/api/v2#section/)

---

### ⚙️ Load the API Key in Python

Make sure you've saved your `MAL_CLIENT_ID` as an environment variable.



In [ ]:
import os

# Load the MyAnimeList Client ID from environment variables
MAL_API_KEY = os.getenv("MAL_API_KEY")

# Optional: show error if not set
if not MAL_API_KEY:
    raise ValueError("❌ MAL_CLIENT_ID not found. Please set it as an environment variable.")


## 🌍 Setting the Search Country

To check streaming availability using **JustWatch**, we need to specify the country where the search will be performed.

This is done by setting a constant with the [two-letter country code (ISO 3166-1 alpha-2)](https://en.wikipedia.org/wiki/List_of_ISO_3166_country_codes).
For example:
- `US` → United States
- `ES` → Spain
- `JP` → Japan
- `BR` → Brazil

### ✏️ Set Your Country Code



In [ ]:
# Set your country code for JustWatch
# Example: 'ES' for Spain
COUNTRY = "ES"